<a href="https://colab.research.google.com/github/brundakushi18-pixel/Cyclone_Prediction_Project/blob/main/Cyclone_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet seaborn
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
sns.set(style="whitegrid", font_scale=1.05)
print("✅ Libraries loaded successfully!")

In [ ]:
from google.colab import files, drive
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"✅ '{filename}' uploaded successfully!")

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded[filename]))
print("✅ Dataset loaded successfully!")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()


In [ ]:
df.columns = df.columns.str.strip()
df.fillna(df.mean(numeric_only=True), inplace=True)
if 'Latitude' in df.columns:
    df.rename(columns={'Latitude':'LAT'}, inplace=True)
print("Cleaned columns:", df.columns.tolist())
print("Missing values:\n", df.isna().sum().head(5))

In [ ]:
if 'Cyclone_Formation' in df.columns:
    print("\nCyclone distribution (0=No, 1=Yes):")
    print(df['Cyclone_Formation'].value_counts())
    print("\nSummary statistics:\n", df.describe())

In [ ]:
features = ['Sea_Surface_Temperature_C', 'Atmospheric_Pressure_hPa', 'Humidity_pct',
            'Wind_Shear_ms', 'Vorticity_s^-1', 'Ocean_Depth_m',
            'Proximity_to_Coast_km', 'Pre_Existing_Disturbance']
plt.figure(figsize=(15,10))
for i, col in enumerate(features, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(x='Cyclone_Formation', y=col, data=df)
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
target = 'Pre_Existing_Disturbance'
X = df.drop(columns=[target])
y = df[target]
X = pd.get_dummies(X, drop_first=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
print("✅ Data split complete.")

In [ ]:
model = RandomForestClassifier(
n_estimators=400, max_depth=16,
min_samples_split=3, min_samples_leaf=1,
max_features='sqrt', random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]
print("\n🔹 Predicted Cyclone Outcomes (0=No,1=Yes):", y_pred[:15])
print("🔹 Predicted Probabilities:", np.round(y_prob[:10],2))
acc = accuracy_score(y_test, y_pred)
print(f"\n📊 Model Accuracy: {acc*100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test,y_pred, target_names=['No Pre-Existing', 'Pre-Existing']))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm, display_labels=['0(No)','1(Yes)']).plot(cmap='Blues')
plt.title("Confusion Matrix – Cyclone Prediction")
plt.show()
plt.figure(figsize=(3,2))
plt.scatter(range(len(y_test)), y_test, label='Actual', color='green', alpha=0.6)
plt.scatter(range(len(y_pred)), y_pred, label='Predicted', color='orange', alpha=0.6)
plt.yticks([0,1],['0(No)','1(Yes)'])
plt.legend(); plt.grid(True)
plt.title("Actual vs Predicted Cyclone Occurrence")
plt.show()
plt.hist(y_prob,bins=12,color='skyblue',edgecolor='black')
plt.title("Cyclone Probability Distribution")
plt.xlabel("Probability of Cyclone Next Month"); plt.ylabel("Frequency")
plt.grid(True); plt.show()

In [ ]:
importance = model.feature_importances_
feat_imp = pd.DataFrame({'Feature':X.columns,'Importance':importance}).sort_values(by='Importance', ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(x='Importance',y='Feature',data=feat_imp,palette='viridis')
plt.title("Feature Importance in Cyclone Prediction")
plt.tight_layout();
plt.show()
print("FeatureImportance Scores:\n")
print(feat_imp)

In [ ]:
# 🌊 Generate realistic synthetic dataset (Oct 2025–Dec 2026)
dates = pd.date_range("2025-10-01", "2026-12-31")
np.random.seed(42)
df = pd.DataFrame({
    'Date': dates.day, 'Month': dates.month, 'Year': dates.year,
    'SeaTemp': np.random.uniform(26, 32, len(dates)),
    'Pressure': np.random.uniform(980, 1015, len(dates)),
    'Humidity': np.random.uniform(60, 95, len(dates)),
    'WindShear': np.random.uniform(5, 25, len(dates)),
    'Vorticity': np.random.uniform(-1.5, 1.5, len(dates)),
    'OceanDepth': np.random.uniform(10, 100, len(dates)),
    'CoastProximity': np.random.uniform(0, 1, len(dates)),
    'Disturbances': np.random.randint(0, 2, len(dates))
})
df['Cyclone'] = ((df.SeaTemp>29)&(df.Pressure<995)&(df.Humidity>85)&(df.WindShear<15)).astype(int)

# 🌪 Train model
X, y = df.drop('Cyclone', axis=1), df['Cyclone']
X = pd.get_dummies(X, drop_first=True)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42).fit(Xtr, ytr)

# 📅 Predict for a given date
try:
    d, m, y = map(int, [input("Date: "), input("Month: "), input("Year: ")])
    sel = df[(df.Year==y)&(df.Month==m)&(df.Date==d)]
    if not sel.empty:
        print(f"\n🌪 Cyclone parameters for {d:02d}-{m:02d}-{y}:\n"); display(sel)
        s = pd.get_dummies(sel.drop('Cyclone', axis=1), drop_first=True).reindex(columns=X.columns, fill_value=0)
        pred, prob = model.predict(s), model.predict_proba(s)[:,1]
        print(f"\n🔹 Prediction (0=No,1=Yes): {pred.tolist()}")
        print(f"🔹 Probability of Cyclone: {np.round(prob*100,2)}%")
    else: print("⚠️ No data found for that date.")
except: print("⚠️ Enter valid numeric Date, Month, and Year.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 📊 Visualize parameters of the selected date
if not sel.empty:
    params = sel.drop(columns=['Year', 'Month', 'Date', 'Cyclone']).iloc[0]

    # 🔹 Apply log scaling to handle large value differences
    log_values = np.log10(params + 1)  # +1 to avoid log(0)

    plt.figure(figsize=(7, 4))
    plt.bar(params.index, log_values, color='skyblue', edgecolor='black')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("Log-scaled Value (log₁₀)")
    plt.title(f"Cyclone Parameters on {d:02d}-{m:02d}-{y}")
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()